In [32]:
# Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pomegranate import *
from scipy.stats import *
from sklearn import svm, linear_model, ensemble, naive_bayes, discriminant_analysis, model_selection, metrics
import lightgbm
import cvxpy as cvx
import random as rnd
import pdb

In [33]:
# Supporting functions

class Distances(object):
    
    def __init__(self,P,Q):
        if sum(P)<1e-20 or sum(Q)<1e-20:
            raise "One or both vector are zero (empty)..."
        if len(P)!=len(Q):
            raise "Arrays need to be of equal sizes..."
        #use numpy arrays for efficient coding
        P=np.array(P,dtype=float);Q=np.array(Q,dtype=float)
        #Correct for zero values
        P[np.where(P<1e-20)]=1e-20
        Q[np.where(Q<1e-20)]=1e-20
        self.P=P
        self.Q=Q
        
    def sqEuclidean(self):
        P=self.P; Q=self.Q; d=len(P)
        return np.sum((P-Q)**2)
    def probsymm(self):
        P=self.P; Q=self.Q; d=len(P)
        return 2*np.sum((P-Q)**2/(P+Q))
    def topsoe(self):
        P=self.P; Q=self.Q
        return np.sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
    def hellinger(self):
        P=self.P; Q=self.Q
        return 2 * np.sqrt(1 - sum(np.sqrt(P * Q)))


def DyS_distance(sc_1, sc_2, measure='topsoe'):
   
    dist = Distances(sc_1, sc_2)

    if measure == 'sqEuclidean':
        return dist.sqEuclidean()
    if measure == 'topsoe':
        return dist.topsoe()
    if measure == 'probsymm':
        return dist.probsymm()
    if measure == 'hellinger':
        return dist.hellinger()
    print("Error, unknown distance specified, returning topsoe")
    return dist.topsoe()


def TernarySearch(left, right, f, eps=1e-4):

    while True:
        if abs(left - right) < eps:
            return (left + right) / 2, f((left + right) / 2)
    
        leftThird  = left + (right - left) / 3
        rightThird = right - (right - left) / 3
    
        if f(leftThird) > f(rightThird):
            left = leftThird
        else:
            right = rightThird 


def getHist(scores, nbins):
    breaks = np.linspace(0, 1, int(nbins)+1)
    breaks = np.delete(breaks, -1)
    breaks = np.append(breaks,1.1)
    
    re = np.repeat(1/(len(breaks)-1), (len(breaks)-1))  
    for i in range(1,len(breaks)):
        re[i-1] = (re[i-1] + len(np.where((scores >= breaks[i-1]) & (scores < breaks[i]))[0]) ) / (len(scores)+1)
    return re

def class_dist(Y, nclasses):
    return np.array([np.count_nonzero(Y == i) for i in range(nclasses)]) / Y.shape[0]

def class2index(labels, classes):
    return np.array([classes.index(labels[i]) for i in range(labels.shape[0])])

In [34]:
# Base quantifiers

def DyS(pos_scores, neg_scores, test_scores, measure='topose'):
    
    bin_size = np.linspace(2,20,10)  #[10,20] range(10,111,10) #creating bins from 2 to 10 with step size 2
    bin_size = np.append(bin_size, 30)
    
    alphas = np.zeros(len(bin_size))
    dists = np.zeros(len(bin_size))
    for i, bins in enumerate(bin_size):
        #....Creating Histograms bins score\counts for validation and test set...............
        
        p_bin_count = getHist(pos_scores, bins)
        n_bin_count = getHist(neg_scores, bins)
        te_bin_count = getHist(test_scores, bins)
        
        def f(x):            
            return(DyS_distance(((p_bin_count*x) + (n_bin_count*(1-x))), te_bin_count, measure = measure))
    
        alphas[i], dists[i] = TernarySearch(0, 1, f)
    
    return np.median(alphas)
    
def EMQ(test_scores, train_labels, nclasses):
    max_it = 1000        # Max num of iterations
    eps = 1e-6           # Small constant for stopping criterium

    p_tr = class_dist(train_labels, nclasses)
    p_s = np.copy(p_tr)
    p_cond_tr = np.array(test_scores)
    p_cond_s = np.zeros(p_cond_tr.shape)

    for it in range(max_it):
        r = p_s / p_tr
        p_cond_s = p_cond_tr * r
        s = np.sum(p_cond_s, axis = 1)
        for c in range(nclasses):
            p_cond_s[:,c] = p_cond_s[:,c] / s
        p_s_old = np.copy(p_s)
        p_s = np.sum(p_cond_s, axis = 0) / p_cond_s.shape[0]
        if (np.sum(np.abs(p_s - p_s_old)) < eps):
            break

    return(p_s/np.sum(p_s))

def GAC(train_scores, test_scores, train_labels, nclasses):
   
    yt_hat = np.argmax(train_scores, axis = 1)
    y_hat = np.argmax(test_scores, axis = 1)
    CM = metrics.confusion_matrix(train_labels, yt_hat, normalize="true").T
    p_y_hat = np.zeros(nclasses)
    values, counts = np.unique(y_hat, return_counts=True)
    p_y_hat[values] = counts
    p_y_hat = p_y_hat/p_y_hat.sum()
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value

def GPAC(train_scores, test_scores, train_labels, nclasses):

    CM = np.zeros((nclasses, nclasses))
    for i in range(nclasses):
        idx = np.where(train_labels == i)[0]
        CM[i] = np.sum(train_scores[idx], axis=0)
        CM[i] /= np.sum(CM[i])
    CM = CM.T
    p_y_hat = np.sum(test_scores, axis = 0)
    p_y_hat = p_y_hat / np.sum(p_y_hat)
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value

def FM(train_scores, test_scores, train_labels, nclasses):

    CM = np.zeros((nclasses, nclasses))
    y_cts = np.array([np.count_nonzero(train_labels == i) for i in range(nclasses)])
    p_yt = y_cts / train_labels.shape[0]
    for i in range(nclasses):
        idx = np.where(train_labels == i)[0]
        CM[:, i] += np.sum(train_scores[idx] > p_yt, axis=0) 
    CM = CM / y_cts
    p_y_hat = np.sum(test_scores > p_yt, axis = 0) / test_scores.shape[0]
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value

In [35]:
# Ensemble quantifiers

def EnsembleDyS(train_scores, test_scores, train_labels, nmodels, nclasses):
    p_hat = np.zeros((nmodels, nclasses))
    for m in range(nmodels):
        for c in range(nclasses):
            pos_scores = [x[c] for indx,x in enumerate(train_scores[m]) if train_labels[indx] == c]
            neg_scores = [x[c] for indx,x in enumerate(train_scores[m]) if train_labels[indx] != c]
            te_scores = [x[c] for x in test_scores[m]]
            p_hat[m][c] = DyS(pos_scores, neg_scores, te_scores, measure='topsoe')
        p_hat[m] = p_hat[m] / np.sum(p_hat[m])
        
    p = np.median(p_hat, axis = 0)
    return(p/np.sum(p))

def EnsembleEM(test_scores, train_labels, nmodels, nclasses):
    p_hat = np.zeros((nmodels, nclasses))
    for m in range(nmodels):
        pdb.set_trace()
        p_hat[m] = EMQ(test_scores[m], train_labels, nclasses)
    
    p = np.median(p_hat, axis = 0)
    return(p/np.sum(p))

def EnsembleGAC(train_scores, test_scores, train_labels, nmodels, nclasses):
    p_hat = np.zeros((nmodels, nclasses))
    for m in range(nmodels):
        p_hat[m] = GAC(train_scores[m], test_scores[m], train_labels, nclasses)
    
    p = np.median(p_hat, axis = 0)
    return(p/np.sum(p))

def EnsembleGPAC(train_scores, test_scores, train_labels, nmodels, nclasses):
    p_hat = np.zeros((nmodels, nclasses))
    for m in range(nmodels):
        p_hat[m] = GPAC(train_scores[m], test_scores[m], train_labels, nclasses)
    
    p = np.median(p_hat, axis = 0)
    return(p/np.sum(p))

def EnsembleFM(train_scores, test_scores, train_labels, nmodels, nclasses):
    p_hat = np.zeros((nmodels, nclasses))
    for m in range(nmodels):
        p_hat[m] = FM(train_scores[m], test_scores[m], train_labels, nclasses)
    
    p = np.median(p_hat, axis = 0)
    return(p/np.sum(p))

In [40]:
print(print(EnsembleFM(train_scores, test_scores, train_labels, nmodels, 3)))

[0.26982652 0.37636464 0.35380884]
None


In [45]:
nmodels

7

In [5]:
# Model Building functions

def getScores(X_train, X_test, Y_train, nclasses):

    models = [linear_model.LogisticRegression(solver='liblinear', multi_class='ovr'),
              discriminant_analysis.LinearDiscriminantAnalysis(),
              ensemble.RandomForestClassifier(),
              svm.SVC(probability=True),
              lightgbm.LGBMClassifier(),
              naive_bayes.GaussianNB(),
              ensemble.GradientBoostingClassifier()]
   
    train_scores = np.zeros((len(models), len(X_train), nclasses))
    test_scores = np.zeros((len(models), len(X_test), nclasses))
    for i, model in enumerate(models):
       
        Y_cts = np.unique(Y_train, return_counts=True)
        nfolds = min(10, min(Y_cts[1]))
       
        if nfolds > 1:
            kfold = model_selection.StratifiedKFold(n_splits=nfolds, random_state=1, shuffle=True)
            for train, test in kfold.split(X_train, Y_train):
                model.fit(X_train[train], Y_train[train])
                train_scores[i][test] = model.predict_proba(X_train)[test]
       
        model.fit(X_train, Y_train)
        test_scores[i] = model.predict_proba(X_test)
       
        if nfolds < 2:
            train_scores[i] = model.predict_proba(X_train)
            
    return train_scores, test_scores, len(models)

# Test data sampler
# testSampler(test_scores, test_labels, 0, .3, 100, 3) => Sample test_scores and test_labels considering
# class 0 as a positive class with a 30% prevalence. The remaining two negative classes will have random  
# prevalence values in the range 0-70% and the three class prevalence will sum to 100%.

def testSampler(test_scores, test_labels, pos, ppos, nsamples, nclasses):

    sampled_labels = np.zeros(nsamples)
    
    nsamples_class = np.array([np.count_nonzero(test_labels == i) for i in range(nclasses)])
    if (np.count_nonzero(nsamples_class < nsamples) > 0):
        print("testSampler: one or more classes have less than ", nsamples, "examples")
        return 

    neg = 1 - ppos
    prevalences = np.zeros(nclasses)
    prevalences[pos] = ppos
    for i in range(nclasses-1):
        if i != pos:
            if neg * 100 > 0:
                prevalences[i] = rnd.randrange(0, round(neg*100)) / 100
            neg -= prevalences[i]
    if (pos == nclasses-1):
        prevalences[nclasses-2] += neg
    else:
        prevalences[nclasses-1] += neg
        
    print(prevalences)
    sampled_idxs = np.zeros(nsamples).astype(int)
    pos = 0
    for i in range(nclasses):
        nitems = round(prevalences[i] * nsamples)
        idxs = [idx for idx in range(len(test_labels)) if test_labels[idx] == i]
        sampled_idxs[pos : pos+nitems] = rnd.sample(idxs, nitems)
        pos += nitems
    
    sampled_labels = test_labels[sampled_idxs]
    sampled_scores = test_scores[:, sampled_idxs]
    
    return sampled_scores, sampled_labels

In [6]:
testSampler(test_scores, test_labels, 2, .5, 40, 3)

NameError: name 'test_scores' is not defined

In [37]:
#X_train = np.load("scores/SkillCraft/Row2/Class1/X_train.npy")
#X_test = np.load("scores/SkillCraft/Row2/Class1/X_test.npy")
#Y_train = np.load("scores/SkillCraft/Row2/Class1/y_train.npy")
#Y_test = np.load("scores/SkillCraft/Row2/Class1/y_test.npy")

#train_labels = class2index(Y_train, (1,2,3))
#test_labels = class2index(Y_test, (1,2,3))

from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

train_labels = class2index(y_train, (0,1,2))
test_labels = class2index(y_test, (0,1,2))

train_scores, test_scores, nmodels = getScores(X_train, X_test, y_train, 3)

In [18]:
print(class_dist(test_labels, 3))

[0.28947368 0.44736842 0.26315789]


In [30]:
#print(EnsembleDyS(train_scores, test_scores, train_labels, nmodels, 3))
print(EnsembleEM(test_scores, train_labels, nmodels, 3))
#print(EnsembleGAC(train_scores, test_scores, train_labels, nmodels, 3))
#print(EnsembleGPAC(train_scores, test_scores, train_labels, nmodels, 3))
#print(EnsembleFM(train_scores, test_scores, train_labels, nmodels, 3))

> <ipython-input-29-2e77d15a1843>(20)EnsembleEM()
     18     for m in range(nmodels):
     19         pdb.set_trace()
---> 20         p_hat[m] = EMQ(test_scores[m], train_labels, nclasses)
     21 
     22     p = np.median(p_hat, axis = 0)

array([[[1.38170983e-004, 3.50003847e-001, 6.49857982e-001],
        [2.24786027e-002, 5.56101703e-001, 4.21419695e-001],
        [8.37023245e-001, 1.62833279e-001, 1.43476116e-004],
        [1.37521744e-002, 6.22707881e-001, 3.63539944e-001],
        [3.58457246e-003, 4.40125232e-001, 5.56290196e-001],
        [1.71967811e-003, 3.47864307e-001, 6.50416015e-001],
        [6.89394304e-003, 3.02224403e-001, 6.90881654e-001],
        [2.19069400e-003, 3.69094878e-001, 6.28714428e-001],
        [8.63748078e-001, 1.36158126e-001, 9.37959544e-005],
        [4.23273707e-002, 6.96242741e-001, 2.61429889e-001],
        [3.06726004e-002, 8.09829789e-001, 1.59497611e-001],
        [3.65890340e-004, 1.59752739e-001, 8.39881371e-001],
        [8.10703275e-001,

BdbQuit: 

In [27]:
pd.DataFrame(test_scores[0])

,0,1,2
0,0.000138,0.350004,0.649858
1,0.022479,0.556102,0.421420
2,0.837023,0.162833,0.000143
3,0.013752,0.622708,0.363540
4,0.003585,0.440125,0.556290
5,0.001720,0.347864,0.650416
6,0.006894,0.302224,0.690882
7,0.002191,0.369095,0.628714
8,0.863748,0.136158,0.000094
9,0.042327,0.696243,0.261430


In [40]:
X_train = np.load("scores/SkillCraft/Row1/Class1/X_train.npy")
X_test = np.load("scores/SkillCraft/Row1/Class1/X_test.npy")
Y_train = np.load("scores/SkillCraft/Row1/Class1/y_train.npy")
Y_test = np.load("scores/SkillCraft/Row1/Class1/y_test.npy")

train_labels = class2index(Y_train, (1,2,3))
test_labels = class2index(Y_test, (1,2,3))
train_scores, test_scores, nmodels = getScores(X_train, X_test, Y_train, 3)

In [42]:
print(class_dist(test_labels, 3))

[0.09871245 0.70171674 0.19957082]


In [41]:
print(EnsembleDyS(train_scores, test_scores, train_labels, nmodels, 3))
print(EnsembleEM(test_scores, train_labels, nmodels, 3))
print(EnsembleGAC(train_scores, test_scores, train_labels, nmodels, 3))
print(EnsembleGPAC(train_scores, test_scores, train_labels, nmodels, 3))
print(EnsembleFM(train_scores, test_scores, train_labels, nmodels, 3))

[0.17626023 0.56626326 0.25747651]
[0.15345401 0.60479548 0.24175051]
[0.20411278 0.52520812 0.2706791 ]
[0.17980907 0.5762228  0.24396814]
[0.14873601 0.57811581 0.27314819]


In [43]:
train_scores = np.load("Wine/train_scores.npy")
test_scores = np.load("Wine/test_scores.npy")
train_labels = np.load("Wine/train_labels.npy")

FM(train_scores, test_scores, train_labels, 4)

array([2.53628433e-08, 3.33333325e-01, 3.33333325e-01, 3.33333325e-01])